In [ ]:
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from keras.models import Sequential
from keras.callbacks import Callback
from keras.layers import Dense
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.initializers import RandomUniform
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
warnings.simplefilter("ignore")

In [3]:
np.random.seed(1)

In [4]:
train2 = loadtxt('../../data/monks-2.train',
                         delimiter=' ', usecols=range(1, 8))
test2 = loadtxt('../../data/monks-2.test',
                        delimiter=' ', usecols=range(1, 8))

In [5]:
X_train2=train2[:, 1:7]
y_train2=train2[:, 0]

X_test2=test2[:, 1:7]
y_test2=test2[:, 0]

In [6]:
enc=OneHotEncoder()

In [7]:
X_train2=enc.fit_transform(X_train2).toarray()
X_test2=enc.fit_transform(X_test2).toarray()

In [8]:
def create_modelCV(lr=0.1, mom=0.1, act='relu'):
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=17, kernel_initializer=RandomUniform(minval=-0.1, maxval=0.1, seed=1), 
                    activation=act,
                   kernel_regularizer=l2(0.0001)))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer= SGD(lr=lr, momentum=mom, nesterov=False))
    return model

In [ ]:
hyper_params_space = {
        'lr': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
        'mom': [0.1, 0.2, 0.3, 0.5, 0.7, 0.8],
        'act': ['sigmoid', 'relu']
    },



print('===================================')
model = KerasClassifier(build_fn=create_modelCV, batch_size=25, epochs=150)
mlpr = GridSearchCV( model
                    , hyper_params_space, scoring=['accuracy'], refit='accuracy', cv=3, n_jobs=2)
mlpr.fit(X_train2, y_train2)
print("DONE")
resultGSCV=pd.DataFrame(mlpr.cv_results_)

In [ ]:
mlpr.best_params_

In [ ]:
def create_modelOP():
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=17, kernel_initializer=RandomUniform(minval=-0.1, maxval=0.1, seed=1), 
                    activation=mlpr.best_params_['act'],
                   kernel_regularizer=l2(0.0001)))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', metrics=['accuracy'], 
                  optimizer= SGD(lr=mlpr.best_params_['lr'], momentum=mlpr.best_params_['mom']))
    return model

In [ ]:
#instantiate the model and run it
#es = EarlyStopping(monitor='val_loss', patience=30)
#mc = ModelCheckpoint('best_model_NOREG.h5', monitor='val_loss', save_best_only=True)

model1 = create_modelOP()

history1 = model1.fit(X_train2, y_train2, 
                      validation_data=(X_test2, y_test2), 
                      epochs=100, 
                      batch_size=25, 
                      #callbacks=[es,mc]
                     ).history

In [ ]:
#See if we have overfitting
plt.figure(figsize=(5,5))
plt.plot(history1['loss'], label='MSE')
plt.plot(history1['accuracy'], label='Accuracy', linestyle='dashed')
#plt.plot(history1['val_loss'], label='Test')
#plt.title('Learning curves for training', fontsize = 18)
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
#See if we have overfitting
plt.figure(figsize=(5,5))
plt.plot(history1['val_loss'], label='MSE')
plt.plot(history1['val_accuracy'], label='Accuracy', linestyle='dashed')
#plt.plot(history1['val_loss'], label='Test')
#plt.title('Learning curves for test', fontsize = 18)
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
mse, accuracy = model1.evaluate(X_test2, y_test2)

In [ ]:
print("Accuracy on Test", accuracy)
print("MSE on Test", mse)

In [ ]:
mse_tr, accuracy_tr = model1.evaluate(X_train2, y_train2)

In [ ]:
print("Accuracy on Training", accuracy_tr)
print("MSE on Training", mse_tr)